# Optimizing Pandas Data Types
by Kevin Götz

## Table of Contents

1. **Introduction & Relevance** <br/><br/>

2. **Setup** <br/><br/>

3. Safing Space with **Feature Engineering**
    - 3.1 Dropping the Noise
    - 3.2 Parsing Information to new columns <br/><br/>

4. Safing Space with **Downcasting**
    - 4.1 Defining the downcast-function
    - 4.2. batch-import csv + downcasting <br/><br/>

5. Using optimized **I/O file formats** <br/><br/>

6. **Outlook**



## 1. Introduction & Relevance

Pandas provides data structures for in-memory analytics, which makes using pandas to analyze datasets that are larger than memory datasets somewhat tricky. Even datasets that are a sizable fraction of memory become unwieldy, as some pandas operations need to make intermediate copies. With small data (under 100 megabytes), performance is rarely a problem. When we move to larger data (100 megabytes to multiple gigabytes), performance issues can make run times much longer, and cause code to fail entirely due to insufficient memory.

That's what this Notebook is all about: How to shrink your pandas dataframe so it fit's your RAM better - without losing any information.

## 2. Setup
Import the relevant libraries and download the dataset from [Ken Huang's Kaggle](https://www.kaggle.com/kenhuang41/nba-basic-game-data-by-player).

In [96]:
import pandas as pd
from os.path import getsize

pd.set_option('display.max.columns', None)
pd.set_option('display.precision', 2)

path = '.\Data Input\games.csv'

## 3. Safing Space with Feature Engineering
Sometimes it is enough to just have a look at the columns and the information they give. Logical reasoning before technical aspects pays off, in my experience.
- Does this column hold any valuable information?
- Does it contain any noise that I can get rid off?
- Is the information hidden and does it need to be extracted?

Those questions are being asked and answered for the column (GAME_ID) subsequently.

In [97]:
# read the data
df = pd.read_csv(path)

# and have a glance
df.head()

,GAME_ID,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER
0,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Jae Crowder,36.68,6,11,4,9,0,0,1,5,6,1,2,1,1,3,10,16,48,Starter
1,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Mikal Bridges,36.53,6,9,4,6,0,0,1,5,6,0,2,1,0,0,-2,16,48,Starter
2,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Devin Booker,35.30,10,17,1,2,4,7,0,3,3,7,0,1,6,3,-2,25,48,Starter
3,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Chris Paul,29.83,4,13,2,4,1,1,1,3,4,8,0,0,3,2,12,11,48,Starter
4,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Deandre Ayton,28.33,2,8,0,0,0,0,2,7,9,3,1,2,1,3,-1,4,48,Starter


The Dataset looks pretty clean with mostly numerical data, but it also contains GAME_ID as a long URL-string that doesn't seem to make sense at first.

Strings usually seem to take up the most RAM, so let's see.

In [98]:
# Check memory usage in percent to the total memory taken in RAM (within this DataFrame)
# The parameter deep=True for system-level memory consumption (python objects size are otherwise estimated)
print('Memory usage in %:')
round((df.memory_usage(deep=True) / df.memory_usage(deep=True).sum()) * 100, 2)

Memory usage in %:


Index          0.00
GAME_ID       14.34
TEAM          12.33
OPPT          12.33
TEAM_SCORE     1.35
OPPT_SCORE     1.35
RESULT         9.79
SCORE_DIFF     1.35
PLAYER        12.06
MP             1.35
FG             1.35
FGA            1.35
FG3            1.35
FG3A           1.35
FT             1.35
FTA            1.35
ORB            1.35
DRB            1.35
TRB            1.35
AST            1.35
STL            1.35
BLK            1.35
TOV            1.35
PF             1.35
PLUS_MINUS     1.35
PTS            1.35
TOTAL_MINS     1.35
STARTER       10.80
dtype: float64

Indeed, the string columns take up the most memory. We optimize them later on, but first we want to have a look at GAME_ID and whether we can extract any useful information from that column.

In [99]:
# saving the memory consumption of the GAME_ID for comparison later on
GAME_ID_mem = df.memory_usage(deep=True)[1]

In [100]:
# let's have a look at the data types and the total RAM consumption too
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743423 entries, 0 to 743422
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   GAME_ID     743423 non-null  object 
 1   TEAM        743423 non-null  object 
 2   OPPT        743423 non-null  object 
 3   TEAM_SCORE  743423 non-null  int64  
 4   OPPT_SCORE  743423 non-null  int64  
 5   RESULT      743423 non-null  object 
 6   SCORE_DIFF  743423 non-null  int64  
 7   PLAYER      743423 non-null  object 
 8   MP          743423 non-null  float64
 9   FG          743423 non-null  int64  
 10  FGA         743423 non-null  int64  
 11  FG3         743423 non-null  int64  
 12  FG3A        743423 non-null  int64  
 13  FT          743423 non-null  int64  
 14  FTA         743423 non-null  int64  
 15  ORB         743423 non-null  int64  
 16  DRB         743423 non-null  int64  
 17  TRB         743423 non-null  int64  
 18  AST         743423 non-null  int64  
 19  ST

### 3.1 Dropping the Noise
GAME_ID seems to start and end always with the same phrase. If that is the case those parts of the string don't hold any information for us and we can drop them.

In [101]:
# GAME_ID seems to be repetitive, maybe we can break it down --> All 10_000 rows start with the same phrase
print(f'{df.GAME_ID.str.startswith("/boxscores/").value_counts()[1]} of {len(df)} Rows start with the string "/boxscores/"')

# maybe it all ends with the same string as well  --> all ends the same
print(f'{df.GAME_ID.str.endswith(".html").value_counts()[1]} of {len(df)} Rows end with the string ".html"')

743423 of 743423 Rows start with the string "/boxscores/"
743423 of 743423 Rows end with the string ".html"


---> All of the strings start and end the same way! So this is noise and can be dropped.

In [102]:
# so we can chop off the unnecessary characters and make sense of the rest
df['GAME_ID'] = df.GAME_ID.str[11:23]
df.head()

,GAME_ID,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER
0,202012310UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Jae Crowder,36.68,6,11,4,9,0,0,1,5,6,1,2,1,1,3,10,16,48,Starter
1,202012310UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Mikal Bridges,36.53,6,9,4,6,0,0,1,5,6,0,2,1,0,0,-2,16,48,Starter
2,202012310UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Devin Booker,35.30,10,17,1,2,4,7,0,3,3,7,0,1,6,3,-2,25,48,Starter
3,202012310UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Chris Paul,29.83,4,13,2,4,1,1,1,3,4,8,0,0,3,2,12,11,48,Starter
4,202012310UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Deandre Ayton,28.33,2,8,0,0,0,0,2,7,9,3,1,2,1,3,-1,4,48,Starter


### 3.2 Parsing new GAME_ID to more meaningful columns
The rest of the GAME_ID string seems to be the date & location of the game. Let's check that.

In [103]:
# The last 3 characters seem to be the location of the game
df.insert(1, 'LOCATION', df.GAME_ID.str[-3:])

# Check: Every time UTA is the location, Utah Jazz has to be in TEAM or OPPT (because the granularity is player it can be both)
UTA_count = len(df.loc[df["LOCATION"] == "UTA"])
print(f'{UTA_count} times "UTA" in DataFrame.')

UTA_JAZZ_count = len(df.loc[(df["LOCATION"] == "UTA") & ((df["TEAM"] == "Utah Jazz") | (df["OPPT"] == "Utah Jazz"))])
print(f'{UTA_JAZZ_count} "UTA" & "Utah Jazz" in DataFrame.')

25353 times "UTA" in DataFrame.
25353 "UTA" & "Utah Jazz" in DataFrame.


In [104]:
# Rest of the string seems to be a date (except the trailing zero)
# Has the trailing zero any meaning? --> No, because it's the same with all rows
df.GAME_ID.str[-4].value_counts()

0    743423
Name: GAME_ID, dtype: int64

In [105]:
# Adapting the string to only contain the date
df['GAME_ID'] = df.GAME_ID.str[:-4]
df['GAME_ID'].value_counts()

20161125    388
20130417    384
20140416    384
20120418    362
20151028    362
           ... 
19970324     23
20010503     23
20010427     23
20201011     22
20010122     21
Name: GAME_ID, Length: 4860, dtype: int64

In [106]:
# Parsing the string to Date Format
df.insert(0, 'DATE', pd.to_datetime(df['GAME_ID'], yearfirst=True))

# getting rid of the GAME_ID because all relevant information is safed
df.drop(columns='GAME_ID', inplace=True)
df.head()

,DATE,LOCATION,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER
0,2020-12-31,UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Jae Crowder,36.68,6,11,4,9,0,0,1,5,6,1,2,1,1,3,10,16,48,Starter
1,2020-12-31,UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Mikal Bridges,36.53,6,9,4,6,0,0,1,5,6,0,2,1,0,0,-2,16,48,Starter
2,2020-12-31,UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Devin Booker,35.30,10,17,1,2,4,7,0,3,3,7,0,1,6,3,-2,25,48,Starter
3,2020-12-31,UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Chris Paul,29.83,4,13,2,4,1,1,1,3,4,8,0,0,3,2,12,11,48,Starter
4,2020-12-31,UTA,Phoenix Suns,Utah Jazz,106,95,W,11,Deandre Ayton,28.33,2,8,0,0,0,0,2,7,9,3,1,2,1,3,-1,4,48,Starter


--> So with small feature engineering we already safed some memory!

In [107]:
# Compute the memory reduction
DATE_LOCATION_mem = df.memory_usage(deep=True)[1:3].sum()
mem_diff = (1 - (DATE_LOCATION_mem / GAME_ID_mem)) * 100
print(f'The useful information from GAME_ID is now stored in DATE & LOCATION. Memory Reduction: {round(mem_diff)}%.')

The useful information from GAME_ID is now stored in DATE & LOCATION. Memory Reduction: 20%.


## 4. Safing Space with Downcasting

### 4.1 Defining the downcast-function
The function consists of 3 main parts: categorize the strings, downcast the integers, downcast the floats. Apart from that it also returns a print with the amount of the memory reduction.


In [108]:
# taking care of all 3 standard Pandas dtypes

def downcast(df: pd.DataFrame) -> pd.DataFrame:
    
    ''' Compression of the common dtypes "float64", "int64", "object" or "string" '''

    # memory before downcasting
    mem_before = df.memory_usage(deep=True).sum()
    mem_before_mb = round(mem_before / (1024**2), 2)

    # convert the dataframe columns to appropriate dtypes (e.g. object to string, or 1.0 float to 1 integer, etc.)
    df = df.convert_dtypes()

    # string categorization (only the ones with low cardinality)
    for column in df.select_dtypes(['string', 'object']):
        if (len(df[column].unique()) / len(df[column])) < 0.5:
            df[column] = df[column].astype('category')

    # float64 downcasting
    for column in df.select_dtypes(['float']):
        df[column] = pd.to_numeric(df[column], downcast='float')

    # int64 downcasting (depending if negative values are apparent (='signed') or only >=0 (='unsigned'))
    for column in df.select_dtypes(['integer']):
        if df[column].min() >= 0:
            df[column] = pd.to_numeric(df[column], downcast='unsigned')
        else:
            df[column] = pd.to_numeric(df[column], downcast='signed')

    # memory after downcasting & compression
    mem_after = df.memory_usage(deep=True).sum()
    mem_after_mb = round(mem_after / (1024**2), 2)
    compression = round(((mem_before - mem_after) / mem_before) * 100)

    # downcasting summary
    print(f'DataFrame compressed by {compression}% from {mem_before_mb} MB down to {mem_after_mb} MB.')

    return df

### 4.2 Batch-Import csv + Downcasting

We don't want to store the whole DataFrame at once into our RAM. That's why we apply the downcast-function and the feature engineering to every chunk!

In [109]:
# Batch-Import of the Data:
res_chunk = []

# loop through the batches of the csv
for chunk in pd.read_csv(path, chunksize=100_000, iterator=True):

    # The 3 characters are the location of the game
    chunk.insert(1, 'LOCATION', chunk.GAME_ID.str[-8:-5])

    # Parsing the string to Date Format
    chunk.insert(0, 'DATE', chunk.GAME_ID.str[11:19])
    chunk['DATE'] = pd.to_datetime(chunk['DATE'], yearfirst=True)

    # getting rid of the GAME_ID because all relevant information is safed
    chunk.drop(columns='GAME_ID', inplace=True)

    # downcast with the function we build earlier
    downcast(chunk)

    # store the chunk DataFrame in a list
    res_chunk.append(chunk)

DataFrame compressed by 89% from 54.95 MB down to 5.83 MB.
DataFrame compressed by 89% from 54.95 MB down to 5.8 MB.
DataFrame compressed by 89% from 54.88 MB down to 5.8 MB.
DataFrame compressed by 89% from 54.89 MB down to 5.79 MB.
DataFrame compressed by 89% from 54.98 MB down to 5.8 MB.
DataFrame compressed by 89% from 54.89 MB down to 5.79 MB.
DataFrame compressed by 89% from 54.78 MB down to 5.8 MB.
DataFrame compressed by 89% from 23.77 MB down to 2.55 MB.


--> We can see that the downcasting is really successful, reducing RAM usage by 89%!

In [110]:
# Concatenate and Downcast again
df = downcast(pd.concat(res_chunk))

DataFrame compressed by 90% from 408.68 MB down to 42.77 MB.


--> Unfortunately, when we concat the downsized batches (chunks), pandas upcasts the data types again. That's the reason for the last print saying that it downcasted again by 90%. So chunking is no solution for this function.

In [111]:
# let's have a look at the new data types
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743423 entries, 0 to 743422
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   DATE        743423 non-null  datetime64[ns]
 1   LOCATION    743423 non-null  category      
 2   TEAM        743423 non-null  category      
 3   OPPT        743423 non-null  category      
 4   TEAM_SCORE  743423 non-null  UInt8         
 5   OPPT_SCORE  743423 non-null  UInt8         
 6   RESULT      743423 non-null  category      
 7   SCORE_DIFF  743423 non-null  Int8          
 8   PLAYER      743423 non-null  category      
 9   MP          743423 non-null  Float32       
 10  FG          743423 non-null  UInt8         
 11  FGA         743423 non-null  UInt8         
 12  FG3         743423 non-null  UInt8         
 13  FG3A        743423 non-null  UInt8         
 14  FT          743423 non-null  UInt8         
 15  FTA         743423 non-null  UInt8         
 16  OR

In [112]:
# Show the data
df.tail()

,DATE,LOCATION,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER
743418,1996-11-01,MIN,Minnesota Timberwolves,San Antonio Spurs,82,78,W,4,Terry Porter,16.2,3,6,1,1,0,0,0,1,1,1,1,0,2,1,-2,7,48,Reserve
743419,1996-11-01,MIN,Minnesota Timberwolves,San Antonio Spurs,82,78,W,4,Chris Carr,9.73,0,1,0,1,0,0,0,2,2,0,0,1,1,0,0,0,48,Reserve
743420,1996-11-01,MIN,Minnesota Timberwolves,San Antonio Spurs,82,78,W,4,Cherokee Parks,5.32,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-10,0,48,Reserve
743421,1996-11-01,MIN,Minnesota Timberwolves,San Antonio Spurs,82,78,W,4,Shane Heal,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,Reserve
743422,1996-11-01,MIN,Minnesota Timberwolves,San Antonio Spurs,82,78,W,4,Dean Garrett,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,Reserve


## 5. Using optimized I/O file formats


**Advantage of Feather over CSV**:
- saving optimized Data Formats (Downcast permanent)
- better compression (less memory usage: -29% when feather(lz4) compared with csv(gzip))
- faster I/O (parallelizable: >90% faster)

In [113]:
# Saving to Feather File Format and CSV File Format for Comparison
df.to_feather('.\Data Output\games.feather')
df.to_csv('.\Data Output\games_zip.csv')

In [114]:
# Comparing the loading times
%timeit x = pd.read_feather('.\Data Output\games.feather')
%timeit y = pd.read_csv('.\Data Output\games_zip.csv')

89.6 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.59 s ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [115]:
# Feather also takes less disc memory
filesize_diff = (1 - (getsize(".\Data Output\games.feather") / getsize(".\Data Output\games_zip.csv"))) * 100
print(f'.feather uses {round(filesize_diff)}% less disc space compared to .csv')

.feather uses 87% less disc space compared to .csv


## 6. Outlook

There is still one big disadvantage: The function is only applicable when the DataFrame is already loaded into RAM. So what if we need to optimize the data types before loading it into RAM because it doesn't fit in there yet? That's where chunking and saving the minimum and maximum of the column into a DataFrame comes in handy. One could then downcast this intermediate DataFrame and safe the optimized Data Types as a dictionary for the read_csv parameter "dtype: dict". Another opportunity could be a out-of-core library like [Vaex](https://vaex.io/docs/index.html) that makes use of memory-mapping and doesn't load all the data into RAM.

Also sparse arrays weren't used in this project and could be an advantage in certain datasets with sparse data, e.g. tables for recommendation engines that have a lot of 1/0 data. This feature would exceed an acceptable function length and could be added in a future project for a custom module.